In [1]:
# IMPORTS

import platform
import sys
import time
from shutil import copyfile
from tempfile import TemporaryDirectory

import numpy
from sklearn.datasets import load_breast_cancer

from concrete.ml.deployment import FHEModelClient, FHEModelDev, FHEModelServer
from concrete.ml.sklearn import XGBClassifier

In [2]:
# ENVIROMENT CLASS

class OnDiskenviroment:
    def __init__(self):
        self.server_dir = TemporaryDirectory()  
        self.client_dir = TemporaryDirectory()
        self.dev_dir = TemporaryDirectory()

    def client_send_evaluation_key_to_server(self, serialized_evaluation_keys):
        with open(self.server_dir.name + "/serialized_evaluation_keys.ekl", "wb") as f:
            f.write(serialized_evaluation_keys)

    def client_send_input_to_server_for_prediction(self, input_after_encryption):
        with open(self.server_dir.name + "/serialized_evaluation_keys.ekl", "rb") as f:
            serialized_evaluation_keys = f.read()
        time_begin = time.time()
        encrypted_prediction = FHEModelServer(self.server_dir.name).run(
            input_after_encryption, serialized_evaluation_keys
        )
        time_end = time.time()
        with open(self.server_dir.name + "/encrypted_prediction.enc", "wb") as f:
            f.write(encrypted_prediction)
        return time_end - time_begin

    def dev_send_model_to_server(self):
        copyfile(self.dev_dir.name + "/server.zip", self.server_dir.name + "/server.zip")

    def server_send_encrypted_prediction_to_client(self):
        with open(self.server_dir.name + "/encrypted_prediction.enc", "rb") as f:
            encrypted_prediction = f.read()
        return encrypted_prediction

    def dev_send_clientspecs_and_modelspecs_to_client(self):
        copyfile(self.dev_dir.name + "/client.zip", self.client_dir.name + "/client.zip")
        copyfile(
            self.dev_dir.name + "/serialized_processing.json",
            self.client_dir.name + "/serialized_processing.json",
        )

    def cleanup(self):
        self.server_dir.cleanup()
        self.client_dir.cleanup()
        self.dev_dir.cleanup()

In [3]:
# MODEL PREPARATION TO FHE AND ENVIROMENT

X, y = load_breast_cancer(return_X_y=True)

X_model_owner, X_client = X, X
y_model_owner, y_client = y, y

n_estimators = 10

model_to_fhe = XGBClassifier(n_bits=2, n_estimators=n_estimators, max_depth=3)
model_to_fhe.fit(X_model_owner, y_model_owner)
model_to_fhe.compile(X_model_owner)

enviroment = OnDiskenviroment()

fhemodel_to_fhe = FHEModelDev(enviroment.dev_dir.name, model_to_fhe)
fhemodel_to_fhe.save()


In [4]:
# SENDING APPROPRIATE DATA TO ENVIROMENT 

!ls -lh $enviroment.dev_dir.name
enviroment.dev_send_model_to_server()
!ls -lh $enviroment.server_dir.name
enviroment.dev_send_clientspecs_and_modelspecs_to_client()
!ls -lh $enviroment.client_dir.name

total 60K
-rw-rw-r-- 1 cybant cybant  464 lis 16 10:35 client.zip
-rw-rw-r-- 1 cybant cybant  44K lis 16 10:35 serialized_processing.json
-rw-rw-r-- 1 cybant cybant 8,3K lis 16 10:35 server.zip
total 12K
-rw-rw-r-- 1 cybant cybant 8,3K lis 16 10:35 server.zip
total 48K
-rw-rw-r-- 1 cybant cybant 464 lis 16 10:35 client.zip
-rw-rw-r-- 1 cybant cybant 44K lis 16 10:35 serialized_processing.json


In [5]:
# CLIENT PREPERATION - KEYS

client_model = FHEModelClient(enviroment.client_dir.name, key_dir=enviroment.client_dir.name)
client_model.generate_private_and_evaluation_keys()

serialized_evaluation_keys = client_model.get_serialized_evaluation_keys()

print(f"Evaluation keys size: {sys.getsizeof(serialized_evaluation_keys) / 1024 / 1024:.2f} MB")
enviroment.client_send_evaluation_key_to_server(serialized_evaluation_keys)

Evaluation keys size: 119.88 MB


KeySetCache: miss, regenerating /tmp/tmpdy54jji6/7071120041192343400/0_0


In [6]:
# MAIN LOOP WHICH ENRYPT DATA AND MAKE STATISTICS

decrypted_predictions = []
time_exec = []
for i in range(X_client.shape[0]):
    if i % 20 == 0:
        print("Step " + str(i)  + " done")
    buffor = X_client[[i], :]
    input_after_encryption = client_model.quantize_encrypt_serialize(buffor)
    time_exec += [enviroment.client_send_input_to_server_for_prediction(input_after_encryption)]
    encrypted_prediction = enviroment.server_send_encrypted_prediction_to_client()
    decrypted_prediction = client_model.deserialize_decrypt_dequantize(encrypted_prediction)[0]
    decrypted_predictions.append(decrypted_prediction)

clear_prediction_classes = model_to_fhe.predict_proba(X_client).argmax(axis=1)
decrypted_predictions_classes = numpy.array(decrypted_predictions).argmax(axis=1)
accuracy = (clear_prediction_classes == decrypted_predictions_classes).mean()



Step 0 done
Step 20 done
Step 40 done
Step 60 done
Step 80 done
Step 100 done
Step 120 done
Step 140 done
Step 160 done
Step 180 done
Step 200 done
Step 220 done
Step 240 done
Step 260 done
Step 280 done
Step 300 done
Step 320 done
Step 340 done
Step 360 done
Step 380 done
Step 400 done
Step 420 done
Step 440 done
Step 460 done
Step 480 done
Step 500 done
Step 520 done
Step 540 done
Step 560 done


In [7]:
# INTERESTING PRINTOUTS
print(f"Encrypted data is "f"{sys.getsizeof(input_after_encryption)/sys.getsizeof(buffor):.2f}"" larger than data")
print(f"Execution time are {[numpy.round(e, 2) for e in time_exec]}, ie an average of " f"{numpy.mean(time_exec):.2f} seconds")
print(f"Accuracy between FHE prediction and clear model is: {accuracy*100:.0f}%")

Encrypted data is 1336.48 larger than data
Execution time are [8.43, 7.82, 7.51, 7.66, 7.66, 7.84, 7.78, 8.27, 7.72, 316.8, 6.15, 5.7, 5.84, 6.1, 7.64, 8.22, 7.72, 7.53, 7.76, 7.87, 7.82, 10.25, 14.81, 14.46, 14.98, 15.57, 15.54, 14.52, 15.17, 14.55, 13.58, 14.72, 14.34, 14.65, 15.22, 15.54, 14.93, 14.46, 14.13, 14.47, 14.88, 15.8, 13.72, 13.6, 14.97, 15.08, 15.68, 15.27, 15.85, 16.16, 14.76, 14.56, 14.91, 15.25, 15.15, 15.66, 15.42, 16.32, 14.12, 14.64, 13.75, 14.55, 14.83, 14.78, 14.85, 14.84, 15.08, 13.68, 14.33, 14.75, 14.57, 14.51, 14.93, 14.56, 15.4, 15.26, 15.27, 14.55, 15.02, 14.41, 14.37, 14.32, 14.36, 14.69, 14.39, 14.65, 14.69, 15.17, 14.65, 14.85, 14.98, 14.75, 14.5, 14.36, 14.55, 14.0, 13.33, 14.51, 14.23, 14.81, 14.55, 13.74, 14.06, 13.82, 13.99, 13.57, 14.2, 13.77, 14.49, 14.63, 15.43, 14.91, 14.76, 14.62, 14.68, 15.32, 15.25, 14.75, 14.36, 15.21, 14.13, 14.24, 14.38, 14.02, 14.47, 14.45, 14.68, 14.2, 14.87, 14.24, 14.55, 14.06, 13.65, 14.69, 14.62, 15.18, 15.06, 14.81, 